In [47]:
from newspaper import Article
import nltk

url = 'https://www.yahoo.com/news/labor-board-accuses-apple-suppressing-222846876.html'
article = Article(url)

try:
    article.download()
    article.parse()
    article.nlp()  # Run NLP after parsing

    # Print results
    print("Summary:", article.summary)
except Exception as e:
    print("An error occurred:", e)

Summary: The National Labor Relations Board last week accused Apple of trying to prevent employees from discussing pay equity and pressuring an engineer who attempted to circulate a survey about wages to quit.
These actions by the company, the complaint said, unlawfully forced Scarlett to quit.
Apple managers allegedly threatened other employees who posted on social media and in Slack and spoke to the press about workplace concerns, according to the complaint.
The NLRB is also looking to force Apple to reinstate Scarlett, compensate her for lost pay and issue an apology letter.
The complaints by federal regulators highlight ongoing turmoil around organizing efforts by Apple employees both at the iPhone maker's corporate headquarters and at retail stores.


In [51]:
import newspaper
import pandas as pd

In [57]:
def scrape_tesla_news(url):
    """
    Scrapes Tesla news from the given Yahoo Finance URL.

    Args:
        url: The URL of the Yahoo Finance page for Tesla news.

    Returns:
        A pandas DataFrame with columns 'title', 'summary', and 'date'.
    """

    paper = newspaper.build(url)
    articles = paper.articles

    data = []
    n = 0
    for article in articles:
        if n > 30:
            break
        article.download()
        article.parse()
        article.nlp()
        data.append({'title': article.title,
                     'summary': article.summary,
                     'date': article.publish_date})
        print(article.publish_date)
        n += 1

    df = pd.DataFrame(data)
    return df

In [59]:
# Replace with the actual Yahoo Finance URL for Tesla news
url = "https://finance.yahoo.com/news/tesla"

df = scrape_tesla_news(url)

# Save the DataFrame to a CSV file
df.to_csv('tesla_news.csv', index=False)

print(df)

None
None
None
None
None


ArticleException: Article `download()` failed with HTTPSConnectionPool(host='tw.news.yahoo.com', port=443): Max retries exceeded with url: /%E5%B0%8F%E4%BA%8C%E6%95%B8%E5%AD%B8%E8%80%83%E5%80%92%E5%AE%B6%E9%95%B7%E4%BA%86-6-9-54-%E8%A2%AB%E6%89%A3%E5%88%86-080821085.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B905474AA0>, 'Connection to tw.news.yahoo.com timed out. (connect timeout=7)')) on URL https://tw.news.yahoo.com/%E5%B0%8F%E4%BA%8C%E6%95%B8%E5%AD%B8%E8%80%83%E5%80%92%E5%AE%B6%E9%95%B7%E4%BA%86-6-9-54-%E8%A2%AB%E6%89%A3%E5%88%86-080821085.html